### Setup database connectivity

We'll reuse our module from the previous notebook (***`00_database_connectivity_setup.ipynb`***) to establish connectivity to the database

In [1]:
%run '00_database_connectivity_setup.ipynb'
IPython.display.clear_output()

### Apache MADlib

Model parallel problems or task parallel problems typically involve building a machine learning model on a dataset that cannot fit into memory, on a distributed cluster. The [Apache MADlib](http://madlib.apache.org/) project explicitly parallelizes such models by splitting them into sub-tasks that can be simultaneously executed on multiple nodes of a cluster and combining the results from these sub-tasks to fit the original model.

MADlib has a very rich collection of machine learning algorithms implemented in-database and is highly performant on large scale datasets.

![Apache MADlib](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_1.png?raw=true)

### Breadth of algorithms

![Apache MADlib User Guide](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_2.png?raw=true)

### Architecture

![Apache MADlib Architecture](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_3.png?raw=true)

### Ridge Regression - Demo